In [3]:
!pip install requests beautifulsoup4 pandas


  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/7 [urllib3]
   ----------- ---------------------------- 2/7 [idna]
   ----------------- ---------------------- 3/7 [charset_normalizer]
   ---------------------------- ----------- 5/7 [requests]
   ---------------------------------------- 7/7 [beautifulsoup4]



In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fct  de scrapping

#TODO : to be updated

def scrape_reviews(url, num_pages=1):
    all_reviews = []
    for page in range(1, num_pages + 1):
        page_url = f"{url}?page={page}" # Adapter l'URL de pagination
        print(f"Scraping page: {page_url}")
        try:
            response = requests.get(page_url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status() # Lève une exception pour les codes d'état HTTP d'erreur
            soup = BeautifulSoup(response.text, 'html.parser')

            # Exemple de sélecteurs - À ADAPTER ABSOLUMENT à la structure du site cible
            review_containers = soup.find_all('div', class_='review-item') # Ex: div avec classe 'review-item'

            for container in review_containers:
                review_text = container.find('p', class_='review-text').get_text(strip=True) if container.find('p', class_='review-text') else "N/A"
                rating = container.find('span', class_='rating').get_text(strip=True) if container.find('span', class_='rating') else "N/A"
                # Ajoutez d'autres champs comme date, auteur, etc.

                all_reviews.append({
                    'review_text': review_text,
                    'rating': rating
                })
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête vers {page_url}: {e}")
        time.sleep(2) # Soyez poli avec le serveur, attendez un peu entre les requêtes

    return pd.DataFrame(all_reviews)

# Exemple d'utilisation (URL à remplacer par celle du site tunisien cible)
# url_cible = "https://www.mytek.tn/smartphone-samsung-galaxy-a54-5g-8-256go.html#reviews"
# df_reviews = scrape_reviews(url_cible, num_pages=5)
# df_reviews.to_csv('raw_customer_reviews.csv', index=False)
# print(df_reviews.head())


#chat version

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_tunisianet_reviews(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(product_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Bloc contenant les avis
    reviews_section = soup.find("div", id="product_reviews_block")

    if not reviews_section:
        print("❌ Aucun avis trouvé sur cette page.")
        return pd.DataFrame()

    all_reviews = reviews_section.find_all("div", class_="review-item")
    
    data = []

    for review in all_reviews:

        # Nom du client
        name_tag = review.find("span", class_="reviewer-name")
        name = name_tag.text.strip() if name_tag else "Inconnu"

        # Date de l'avis
        date_tag = review.find("span", class_="review-date")
        date = date_tag.text.strip() if date_tag else "Inconnue"

        # Note (nombre d’étoiles)
        star_container = review.find("div", class_="star_content")
        rating = len(star_container.find_all("i", class_="fa fa-star")) if star_container else None

        # Commentaire
        comment_tag = review.find("p", class_="review-text")
        comment = comment_tag.text.strip() if comment_tag else ""

        # Sauvegarde
        data.append({
            "name": name,
            "date": date,
            "rating": rating,
            "comment": comment
        })

    return pd.DataFrame(data)


# -------------------------------
# 🔥 Tester avec le Redmi Note 13
# -------------------------------

#url = "https://www.tunisianet.com.tn/smartphone/71310-smartphone-xiaomi-redmi-note-13-8-go-256-go-noir.html"
url = "https://www.tunisianet.com.tn/smartphone/71347-smartphone-samsung-galaxy-a54-5g-128-go-8-go-ram-lime.html"

df = scrape_tunisianet_reviews(url)

print(df)

df.to_csv("samsung_a54.csv", index=False)
print("✔️ CSV enregistré ")


❌ Aucun avis trouvé sur cette page.
Empty DataFrame
Columns: []
Index: []
✔️ CSV enregistré 


Code de scraping pour Orange Tunisie (HTML statique)

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_orange_reviews(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(product_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Section des avis
    review_blocks = soup.find_all("div", class_="review")  # à adapter selon le site

    if not review_blocks:
        print("❌ Aucun avis trouvé.")
        return pd.DataFrame()

    data = []

    for review in review_blocks:
        # Nom
        name_tag = review.find("span", class_="reviewer-name")
        name = name_tag.text.strip() if name_tag else "Inconnu"

        # Note
        rating_tag = review.find("div", class_="stars")  # vérifier le HTML exact
        rating = rating_tag["data-rating"] if rating_tag else None

        # Commentaire
        comment_tag = review.find("p", class_="review-text")
        comment = comment_tag.text.strip() if comment_tag else ""

        data.append({
            "name": name,
            "rating": rating,
            "comment": comment
        })

    df = pd.DataFrame(data)
    return df

# -------------------------
# Tester
# -------------------------
url = "https://www.orange.tn/fr/smartphones/samsung-galaxy-a54"
df = scrape_orange_reviews(url)
print(df.head())

df.to_csv("reviews_orange_galaxy_a54.csv", index=False)
print("✅ CSV sauvegardé.")


❌ Aucun avis trouvé.
Empty DataFrame
Columns: []
Index: []
✅ CSV sauvegardé.


# =>
Le problème rencontré avec Beautiful Soup est typique car de nombreux sites, y compris ceux comme Orange et Ooredoo en Tunisie, masquent les avis clients via des techniques dynamiques (JavaScript, chargement asynchrone, données JSON cachées) qui ne sont pas accessibles par un simple parsing HTML.

Solutions pour contourner ce problème
Extraction via données JSON cachées : Certains sites stockent les avis dans des structures JSON invisibles directement dans le code source, qui peuvent être extraites après avoir identifié les bons endpoints ou structures. Par exemple, des sites comme Tripadvisor utilisent cette méthode pour leurs avis clients.

Scraping dynamique avec Selenium ou Puppeteer : Ces outils simulent un navigateur complet qui exécute le JavaScript afin de charger entièrement la page et ses avis avant extraction.

Utilisation d'APIs ou sources alternatives : Parfois, les plateformes ont des APIs publiques ou semi-publiques pour accéder aux avis clients, ou bien on peut extraire les avis depuis des sites tiers comme Trustpilot qui référencent les avis des clients pour ces opérateurs.

Automatisation via plateformes comme Make.com + Apify : Certaines plateformes proposent des modules déjà prêts pour collecter des avis de sites comme Trustpilot, avec des options d'analyse, ce qui pourrait faciliter la collecte.

Exemples spécifiques aux opérateurs en Tunisie
Avis pour Orange Tunisie peuvent être trouvés sur Tripadvisor et Trustpilot mais pas facilement scrapés via BeautifulSoup à cause du contenu dynamique.​

Avis pour Ooredoo Tunisie existent aussi sur Trustpilot et Indeed mais de même, nécessitent des méthodes avancées de scraping ou API.​

Tutoriels sur le scraping d'avis clients montrent que juste BeautifulSoup est souvent insuffisant pour scraper tous les avis car il faut gérer la partie JavaScript ou JSON cachée.​

Pour notre mini-projet on prefere opter pour  : notamment Selenium pour le scraping dynamique, ou bien l'extraction directe de JSON cachés, ou encore d'utiliser des sources comme Trustpilot pour l'analyse des avis clients.

option 1:Selenium

Simule un vrai navigateur

Charge le JS → les avis deviennent visibles

Compatible avec Tunisianet / Orange / Ooredoo

Cette méthode est la plus sûre pour scraper des avis en Tunisie aujourd’hui



Option 2 : Extraire des avis depuis les réseaux sociaux ou forums

Facebook, Twitter, forums de tech tunisiens

Souvent visibles sans JS compliqué

Mais nécessite un peu de nettoyage

I WILL WORK ON BUSINESS HOTEL

MAPS:

https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu&g_ep=EgoyMDI1MTEyMy4xIKXMDSoASAFQAw%3D%3D


avis dans site de booking:
https://www.booking.com/reviews/tn/hotel/business.fr.html?label=gen173nr-10CA0o4gFCCGJ1c2luZXNzSDNYBGjiAYgBAZgBM7gBF8gBDNgBA-gBAfgBAYgCAagCAbgCl86gyQbAAgHSAiRlZjRhOGM4ZS1hNzA0LTRkYzItYjIyNC1hNWMxMjk2YWI0YmXYAgHgAgE&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page=1&r_lang=fr&rows=75&

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape Booking.com reviews
def scrape_booking(url_base, pages=2):
    reviews = []
    for page in range(1, pages + 1):
        url = f"{url_base}&page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch Booking.com page {page}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.find_all('div', class_='review_list_new_item_block')
        for block in review_blocks:
            try:
                reviewer = block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous'
                date = block.find('p', class_='review_item_date').text.strip() if block.find('p', class_='review_item_date') else 'N/A'
                rating = block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A'
                title = block.find('h3', class_='c-review-block__title').text.strip() if block.find('h3', class_='c-review-block__title') else 'No Title'
                text_elements = block.find_all('span', class_='c-review__body')
                text = ' '.join([elem.text.strip() for elem in text_elements]) if text_elements else 'No Text'
                reviews.append({
                    'source': 'Booking.com',
                    'reviewer': reviewer,
                    'date': date,
                    'rating': rating,
                    'title': title,
                    'text': text
                })
            except Exception as e:
                print(f"Error parsing Booking.com review: {e}")
    return reviews

# Function to scrape TripAdvisor reviews
def scrape_tripadvisor(url_base, pages=2):
    reviews = []
    for page in range(0, pages * 10, 10):  # TripAdvisor uses offsets like or10, or20
        url = url_base if page == 0 else f"{url_base}-or{page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch TripAdvisor page {page//10 + 1}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.find_all('div', class_='review-container')
        for block in review_blocks:
            try:
                reviewer = block.find('div', class_='member_info').find('div', class_='info_text').text.strip() if block.find('div', class_='member_info') else 'Anonymous'
                date = block.find('span', class_='ratingDate')['title'] if block.find('span', class_='ratingDate') else 'N/A'
                rating_class = block.find('span', class_='ui_bubble_rating')['class'][1] if block.find('span', class_='ui_bubble_rating') else ''
                rating = str(int(rating_class.replace('bubble_', '')) / 10) if rating_class else 'N/A'
                title = block.find('span', class_='noQuotes').text.strip() if block.find('span', class_='noQuotes') else 'No Title'
                text = block.find('p', class_='partial_entry').text.strip() if block.find('p', class_='partial_entry') else 'No Text'
                reviews.append({
                    'source': 'TripAdvisor',
                    'reviewer': reviewer,
                    'date': date,
                    'rating': rating,
                    'title': title,
                    'text': text
                })
            except Exception as e:
                print(f"Error parsing TripAdvisor review: {e}")
    return reviews

# Function to save reviews to CSV
def save_to_csv(reviews, filename='hotel_reviews.csv'):
    if not reviews:
        print("No reviews to save.")
        return
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(reviews)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs from provided list (use English Booking.com for simplicity; adjust if needed)
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    
    booking_reviews = scrape_booking(booking_url)
    tripadvisor_reviews = scrape_tripadvisor(tripadvisor_url)
    
    all_reviews = booking_reviews + tripadvisor_reviews
    save_to_csv(all_reviews)

Failed to fetch TripAdvisor page 1
Failed to fetch TripAdvisor page 2
No reviews to save.


In [2]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Headers to mimic browser
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    for page in range(1, pages + 1):
        paginated_url = f"{url}&page={page}" if 'booking' in url.lower() else url  # Adjust for site
        response = requests.get(paginated_url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Failed to fetch {source} page {page}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.select(review_selector)
        for block in review_blocks:
            try:
                review = parse_func(block)
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e}")
    return reviews

# Parse function for Booking.com
def parse_booking(block):
    return {
        'reviewer': block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous',
        'date': block.find('p', class_='review_item_date').text.strip() if block.find('p', class_='review_item_date') else 'N/A',
        'rating': block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A',
        'title': block.find('h3', class_='c-review-block__title').text.strip() if block.find('h3', class_='c-review-block__title') else 'No Title',
        'text': ' '.join([elem.text.strip() for elem in block.find_all('span', class_='c-review__body')]) if block.find_all('span', class_='c-review__body') else 'No Text'
    }

# Parse function for Momondo (aggregated pros/cons)
def parse_momondo(block):
    # Momondo has sections for pros/cons; adapt to extract as 'reviews'
    title = block.find('h3').text.strip() if block.find('h3') else 'No Title'  # e.g., 'Pros' or 'Cons'
    items = [li.text.strip() for li in block.find_all('li')]
    return {
        'reviewer': 'Aggregated',
        'date': 'N/A',
        'rating': 'N/A',
        'title': title,
        'text': '; '.join(items)
    }

# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True  # Run without GUI
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)  # Wait for load
    
    for _ in range(pages):
        try:
            review_blocks = driver.find_elements(*review_locator)
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
            # Click next page if exists (adapt selector per site)
            next_button = driver.find_element(By.CSS_SELECTOR, '[data-next-page]') if source == 'TripAdvisor' else None  # Example; customize
            if next_button:
                next_button.click()
                time.sleep(3)
            else:
                break
        except Exception as e:
            print(f"Failed to process {source} page: {e}")
            break
    driver.quit()
    return reviews

# Parse function for TripAdvisor (Selenium)
def parse_tripadvisor(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.info_text').text.strip() if block.find_elements(By.CSS_SELECTOR, '.info_text') else 'Anonymous',
        'date': block.get_attribute('title') if block.find_elements(By.CSS_SELECTOR, '.ratingDate') else 'N/A',
        'rating': str(int(block.get_attribute('class').split('_')[1]) / 10) if block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating') else 'N/A',
        'title': block.find_element(By.CSS_SELECTOR, '.noQuotes').text.strip() if block.find_elements(By.CSS_SELECTOR, '.noQuotes') else 'No Title',
        'text': block.find_element(By.CSS_SELECTOR, '.partial_entry').text.strip() if block.find_elements(By.CSS_SELECTOR, '.partial_entry') else 'No Text'
    }

# Parse function for Google Maps (Selenium)
def parse_google(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.d4r55').text.strip() if block.find_elements(By.CSS_SELECTOR, '.d4r55') else 'Anonymous',  # Adapt based on inspection
        'date': block.find_element(By.CSS_SELECTOR, '.rsqaWe').text.strip() if block.find_elements(By.CSS_SELECTOR, '.rsqaWe') else 'N/A',
        'rating': block.get_attribute('aria-label') if block.find_elements(By.CSS_SELECTOR, '.kvMYJc') else 'N/A',  # Star rating
        'title': 'No Title',  # Google often lacks titles
        'text': block.find_element(By.CSS_SELECTOR, '.wiI7pd').text.strip() if block.find_elements(By.CSS_SELECTOR, '.wiI7pd') else 'No Text'
    }

# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews.csv'):
    if not reviews:
        print("No reviews to save.")
        return
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, keys)
        writer.writeheader()
        writer.writerows(reviews)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs (use your provided ones; English Booking for simplicity)
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    # Scrape static sites
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.review_list_new_item_block', parse_booking)
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', 'div#reviews ul', parse_momondo)  # Adjust selector to target pros/cons lists
    
    # Scrape dynamic sites
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CLASS_NAME, 'review-container'), parse_tripadvisor)
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CLASS_NAME, 'jftiEf'), parse_google)  # Google review class; inspect for accuracy
    
    all_reviews = booking_reviews + momondo_reviews + tripadvisor_reviews + google_reviews
    save_to_csv(all_reviews)

Failed to fetch Momondo page 1
Failed to fetch Momondo page 2
Failed to process TripAdvisor page: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-next-page]"}
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xd04103
	0xd04144
	0xb0e71d
	0xb5a03d
	0xb5a41b
	0xba17f2
	0xb7c954
	0xb9ee17
	0xb7c706
	0xb4da30
	0xb4ed54
	0xf757b4
	0xf7098a
	0xd2c392
	0xd1c4c8
	0xd2324d
	0xd0c478
	0xd0c63c
	0xcf67ca
	0x756e7ba9
	0x7719c3ab
	0x7719c32f

Saved 8 reviews to hotel_reviews.csv


In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIGURATION ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9'
}

# --- HELPER: SETUP DRIVER ---
def get_driver():
    options = Options()
    # options.add_argument("--headless=new")  # Uncomment to run invisible. Run visible first to debug!
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"user-agent={HEADERS['User-Agent']}")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# --- 1. STATIC SCRAPING (Booking.com) ---
def scrape_booking(url, pages=1):
    print("--- Scraping Booking.com ---")
    reviews = []
    for page in range(pages):
        # Booking pagination is largely offset based usually
        offset = page * 25
        # Ensure URL handles pagination correctly
        target_url = f"{url};rows=25;offset={offset}"
        
        try:
            response = requests.get(target_url, headers=HEADERS, timeout=10)
            if response.status_code != 200:
                print(f"Blocked or error on page {page+1}")
                break
            
            soup = BeautifulSoup(response.text, 'html.parser')
            # Generic selector for review cards on Booking (changes often, check 'li.review_list_new_item_block')
            review_blocks = soup.select('li.review_list_new_item_block')
            
            if not review_blocks:
                print("No review blocks found (Selectors might need update via Inspect Element).")
                break

            for block in review_blocks:
                reviews.append({
                    'source': 'Booking.com',
                    'reviewer': block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous',
                    'rating': block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A',
                    'text': block.find('span', class_='c-review__body').text.strip() if block.find('span', class_='c-review__body') else 'No Text'
                })
            print(f"Booking: Collected {len(review_blocks)} reviews from page {page+1}")
            time.sleep(random.uniform(2, 5)) # Polite delay
        except Exception as e:
            print(f"Error on Booking: {e}")
            
    return reviews

# --- 2. DYNAMIC SCRAPING (TripAdvisor - Pagination) ---
def scrape_tripadvisor(url, pages=1):
    print("--- Scraping TripAdvisor ---")
    driver = get_driver()
    reviews = []
    
    try:
        driver.get(url)
        time.sleep(3)
        
        for i in range(pages):
            # 1. Expand reviews if "Read more" exists
            try:
                more_btns = driver.find_elements(By.CSS_SELECTOR, "span[data-test-target='review-text'] span.ui_icon")
                for btn in more_btns:
                    driver.execute_script("arguments[0].click();", btn)
                time.sleep(1)
            except: pass

            # 2. Extract Data (Selectors updated for 2024/2025 layouts)
            # TripAdvisor classes are messy (e.g., 'yCeTE', 'yFGwA'). We try to target data-test-target.
            blocks = driver.find_elements(By.CSS_SELECTOR, "div[data-test-target='reviews-tab'] > div")
            
            # Fallback if specific container not found
            if not blocks:
                blocks = driver.find_elements(By.CLASS_NAME, "review-container")

            for block in blocks:
                try:
                    text_elem = block.find_element(By.CSS_SELECTOR, "span[class*='ui_bubble_rating']")
                    rating = text_elem.get_attribute("class").split("_")[-1] # e.g., "50" -> 5.0
                    
                    reviews.append({
                        'source': 'TripAdvisor',
                        'reviewer': 'Traveler',
                        'rating': f"{int(rating)/10}",
                        'text': block.text.replace("\n", " ")[:200] + "..." # Truncated for safety
                    })
                except: continue
            
            print(f"TripAdvisor: Page {i+1} processed.")

            # 3. Next Page
            try:
                # Look for the next button specifically
                next_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Next')] | //a[contains(@aria-label, 'Next page')]")
                if "disabled" in next_btn.get_attribute("class"):
                    break
                next_btn.click()
                time.sleep(random.uniform(3, 6))
            except Exception:
                print("TripAdvisor: No next button found or reached end.")
                break
                
    except Exception as e:
        print(f"TripAdvisor Error: {e}")
    finally:
        driver.quit()
    
    return reviews

# --- 3. DYNAMIC SCRAPING (Google Maps - Infinite Scroll) ---
def scrape_google_maps(url, max_reviews=20):
    print("--- Scraping Google Maps ---")
    driver = get_driver()
    reviews = []
    
    try:
        driver.get(url)
        # Wait for the sidebar to load
        wait = WebDriverWait(driver, 10)
        # Try to find the scrollable container (the sidebar)
        # Note: Class names like 'm6QErb' are obfuscated and change. 
        # We try to find the container with aria-label="Reviews" or similar.
        time.sleep(3)
        
        # Simple scroll logic for the sidebar
        scrollable_div = None
        try:
            # Common selector for the scrollable review list in Google Maps
            scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div[role="feed"]')
        except:
            print("Google Maps: Could not find scrollable feed. Check URL or Selectors.")
            return []

        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        
        while len(reviews) < max_reviews:
            # Parse visible reviews
            blocks = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf')
            
            for block in blocks[len(reviews):]: # Only process new ones
                try:
                    text = block.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
                    rating = block.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute("aria-label")
                    reviews.append({
                        'source': 'Google Maps',
                        'reviewer': 'Google User',
                        'rating': rating,
                        'text': text
                    })
                except: pass
                
            if len(reviews) >= max_reviews: break

            # Scroll down
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", scrollable_div)
            time.sleep(2)
            
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:
                break # End of list
            last_height = new_height
            
    except Exception as e:
        print(f"Google Maps Error: {e}")
    finally:
        driver.quit()
        
    return reviews

# --- MAIN ---
if __name__ == "__main__":
    # VALID URLS (Example: Hotel Africa Tunis)
    booking_url = "https://www.booking.com/hotel/tn/africa-tunis.en-gb.html"
    ta_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d302476-Reviews-El_Mouradi_Hotel_Africa_Tunis-Tunis_Tunis_Governorate.html"
    # Note: Google Maps URLs must be the full map view URL, not the redirect
    google_url = "https://www.google.com/maps/place/El+Mouradi+Hotel+Africa+Tunis/@36.8000678,10.180014,17z/data=!4m12!3m11!1s0x12fd3464506041c9:0x5e0f06f50b4d0810!5m2!4m1!1i2!8m2!3d36.8000678!4d10.1825889!9m1!1b1!16s%2Fg%2F11b6_y_y7_?entry=ttu"

    all_data = []
    
    # 1. Booking
    all_data.extend(scrape_booking(booking_url, pages=1))
    
    # 2. TripAdvisor
    all_data.extend(scrape_tripadvisor(ta_url, pages=1))
    
    # 3. Google Maps
    all_data.extend(scrape_google_maps(google_url, max_reviews=10))

    # Save
    if all_data:
        keys = all_data[0].keys()
        with open('final_reviews.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, keys)
            writer.writeheader()
            writer.writerows(all_data)
        print(f"Successfully saved {len(all_data)} reviews to final_reviews.csv")
    else:
        print("No reviews extracted.")

--- Scraping Booking.com ---
Blocked or error on page 1
--- Scraping TripAdvisor ---
TripAdvisor: Page 1 processed.
TripAdvisor: No next button found or reached end.
--- Scraping Google Maps ---
Google Maps: Could not find scrollable feed. Check URL or Selectors.
No reviews extracted.


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Headers to mimic browser
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    for page in range(1, pages + 1):
        paginated_url = f"{url}&page={page}" if 'booking' in url.lower() else url  # Adjust for site
        response = requests.get(paginated_url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Failed to fetch {source} page {page}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.select(review_selector)
        for block in review_blocks:
            try:
                review = parse_func(soup if source == 'Momondo' else block)  # Pass soup for Momondo as it's aggregated
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e}")
    return reviews

# Parse function for Booking.com (unchanged)
def parse_booking(block):
    return {
        'reviewer': block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous',
        'date': block.find('p', class_='review_item_date').text.strip() if block.find('p', class_='review_item_date') else 'N/A',
        'rating': block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A',
        'title': block.find('h3', class_='c-review-block__title').text.strip() if block.find('h3', class_='c-review-block__title') else 'No Title',
        'text': ' '.join([elem.text.strip() for elem in block.find_all('span', class_='c-review__body')]) if block.find_all('span', class_='c-review__body') else 'No Text'
    }

# Parse function for Momondo (adjusted for aggregated pros/cons using string search)
def parse_momondo(soup):
    reviews = []
    pros_heading = soup.find(string=lambda text: text and 'Pros +' in text)
    if pros_heading:
        pros_list = pros_heading.find_next('ul')
        if pros_list:
            items = [li.text.strip() for li in pros_list.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Pros',
                'text': '; '.join(items)
            })

    cons_heading = soup.find(string=lambda text: text and 'Cons -' in text)
    if cons_heading:
        cons_list = cons_heading.find_next('ul')
        if cons_list:
            items = [li.text.strip() for li in cons_list.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Cons',
                'text': '; '.join(items)
            })
    return reviews[0] if reviews else {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'No pros/cons found'}

# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)  # Initial load

    for page_num in range(pages):
        if source == 'Google Maps':
            # Scroll reviews panel to load more
            try:
                reviews_panel = driver.find_element(By.CSS_SELECTOR, '.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')
                for _ in range(5):  # Scroll multiple times to load
                    driver.execute_script("arguments[0].scrollBy(0, 2000);", reviews_panel)
                    time.sleep(1)
            except:
                print("No reviews panel found on Google Maps")

        try:
            review_blocks = driver.find_elements(*review_locator)
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
            # Pagination
            if source == 'TripAdvisor':
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '.ui_button.nav.next.primary')
                    if next_button.is_enabled():
                        next_button.click()
                        time.sleep(3)
                    else:
                        break
                except:
                    print(f"No next button on TripAdvisor page {page_num + 1}")
                    break
            else:
                break  # No pagination for Google in this setup
        except Exception as e:
            print(f"Failed to process {source} page {page_num + 1}: {e}")
            break
    driver.quit()
    return reviews

# Parse function for TripAdvisor (unchanged)
def parse_tripadvisor(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.info_text').text.strip() if block.find_elements(By.CSS_SELECTOR, '.info_text') else 'Anonymous',
        'date': block.get_attribute('title') if block.find_elements(By.CSS_SELECTOR, '.ratingDate') else 'N/A',
        'rating': str(int(block.get_attribute('class').split('_')[1]) / 10) if block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating') else 'N/A',
        'title': block.find_element(By.CSS_SELECTOR, '.noQuotes').text.strip() if block.find_elements(By.CSS_SELECTOR, '.noQuotes') else 'No Title',
        'text': block.find_element(By.CSS_SELECTOR, '.partial_entry').text.strip() if block.find_elements(By.CSS_SELECTOR, '.partial_entry') else 'No Text'
    }

# Parse function for Google Maps (unchanged, standard selectors)
def parse_google(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.d4r55').text.strip() if block.find_elements(By.CSS_SELECTOR, '.d4r55') else 'Anonymous',
        'date': block.find_element(By.CSS_SELECTOR, '.rsqaWe').text.strip() if block.find_elements(By.CSS_SELECTOR, '.rsqaWe') else 'N/A',
        'rating': block.get_attribute('aria-label') if block.find_elements(By.CSS_SELECTOR, '.kvMYJc') else 'N/A',
        'title': 'No Title',
        'text': block.find_element(By.CSS_SELECTOR, '.wiI7pd').text.strip() if block.find_elements(By.CSS_SELECTOR, '.wiI7pd') else 'No Text'
    }

# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews_3.csv'):
    if not reviews:
        print("No reviews to save.")
        return
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, keys)
        writer.writeheader()
        writer.writerows(reviews)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    # Scrape static sites
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.review_list_new_item_block', parse_booking)
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', '', parse_momondo)  # Empty selector as we pass soup
    
    # Scrape dynamic sites
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CLASS_NAME, 'review-container'), parse_tripadvisor)
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CLASS_NAME, 'jftiEf'), parse_google)
    
    all_reviews = booking_reviews + momondo_reviews + tripadvisor_reviews + google_reviews
    save_to_csv(all_reviews)

Failed to fetch Momondo page 1
Failed to fetch Momondo page 2
No next button on TripAdvisor page 1
Saved 28 reviews to hotel_reviews.csv


In [6]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Headers to mimic browser and handle potential redirects
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9'
}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    if source == 'Momondo':
        pages = 1  # No pagination for Momondo
    for page in range(1, pages + 1):
        paginated_url = f"{url}&page={page}" if 'booking' in url.lower() else url
        response = requests.get(paginated_url, headers=HEADERS, allow_redirects=True)
        print(f"{source} fetch status: {response.status_code}")  # Log status for debugging
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} page {page} (status: {response.status_code})")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        if source == 'Momondo':
            # For Momondo, parse entire soup directly
            parsed_reviews = parse_func(soup)
            for pr in parsed_reviews:
                pr['source'] = source
                reviews.append(pr)
        else:
            review_blocks = soup.select(review_selector)
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
    return reviews

# Parse function for Booking.com (unchanged)
def parse_booking(block):
    return {
        'reviewer': block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous',
        'date': block.find('p', class_='review_item_date').text.strip() if block.find('p', class_='review_item_date') else 'N/A',
        'rating': block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A',
        'title': block.find('h3', class_='c-review-block__title').text.strip() if block.find('h3', class_='c-review-block__title') else 'No Title',
        'text': ' '.join([elem.text.strip() for elem in block.find_all('span', class_='c-review__body')]) if block.find_all('span', class_='c-review__body') else 'No Text'
    }

# Parse function for Momondo (updated with precise selectors)
def parse_momondo(soup):
    reviews = []
    pros_h3 = soup.find('h3', string=lambda text: text and 'Pros +' in text.strip())
    if pros_h3:
        pros_ul = pros_h3.find_next('ul', class_='pros-list')
        if pros_ul:
            items = [li.text.strip() for li in pros_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Pros',
                'text': '; '.join(items)
            })

    cons_h3 = soup.find('h3', string=lambda text: text and 'Cons -' in text.strip())
    if cons_h3:
        cons_ul = cons_h3.find_next('ul', class_='cons-list')
        if cons_ul:
            items = [li.text.strip() for li in cons_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Cons',
                'text': '; '.join(items)
            })
    return reviews if reviews else [{'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'No pros/cons found'}]

# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)  # Longer initial load for stability

    for page_num in range(pages):
        if source == 'Google Maps':
            try:
                reviews_panel = driver.find_element(By.CSS_SELECTOR, '.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')
                for _ in range(10):  # Increased scrolls to load more
                    driver.execute_script("arguments[0].scrollBy(0, 2000);", reviews_panel)
                    time.sleep(1.5)
            except:
                print("No reviews panel found on Google Maps")

        try:
            review_blocks = driver.find_elements(*review_locator)
            if not review_blocks:
                print(f"No reviews found on {source} page {page_num + 1}")
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
            # Pagination
            if source == 'TripAdvisor':
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '.ui_button.nav.next.primary')
                    if next_button and next_button.is_displayed() and next_button.is_enabled():
                        next_button.click()
                        time.sleep(5)
                    else:
                        print(f"No next button or not enabled on TripAdvisor page {page_num + 1}")
                        break
                except:
                    print(f"No next button found on TripAdvisor page {page_num + 1}")
                    break
            else:
                break
        except Exception as e:
            print(f"Failed to process {source} page {page_num + 1}: {e}")
            break
    driver.quit()
    return reviews

# Parse function for TripAdvisor (added robustness)
def parse_tripadvisor(block):
    try:
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.info_text div:first-child')
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'
        date_elem = block.find_elements(By.CSS_SELECTOR, '.ratingDate')
        date = date_elem[0].get_attribute('title') if date_elem else 'N/A'
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating')
        rating_class = rating_elem[0].get_attribute('class') if rating_elem else ''
        rating = str(int(rating_class.split('_')[-1]) / 10) if rating_class else 'N/A'
        title_elem = block.find_elements(By.CSS_SELECTOR, '.noQuotes')
        title = title_elem[0].text.strip() if title_elem else 'No Title'
        text_elem = block.find_elements(By.CSS_SELECTOR, '.partial_entry')
        text = text_elem[0].text.strip() if text_elem else 'No Text'
        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': title,
            'text': text
        }
    except:
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}

# Parse function for Google Maps (unchanged)
def parse_google(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.d4r55').text.strip() if block.find_elements(By.CSS_SELECTOR, '.d4r55') else 'Anonymous',
        'date': block.find_element(By.CSS_SELECTOR, '.rsqaWe').text.strip() if block.find_elements(By.CSS_SELECTOR, '.rsqaWe') else 'N/A',
        'rating': block.get_attribute('aria-label') if block.find_elements(By.CSS_SELECTOR, '.kvMYJc') else 'N/A',
        'title': 'No Title',
        'text': block.find_element(By.CSS_SELECTOR, '.wiI7pd').text.strip() if block.find_elements(By.CSS_SELECTOR, '.wiI7pd') else 'No Text'
    }

# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews___.csv'):
    if not reviews:
        print("No reviews to save.")
        return
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, keys)
        writer.writeheader()
        writer.writerows(reviews)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    # Scrape static sites
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.review_list_new_item_block', parse_booking)
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', '', parse_momondo)  # Empty selector for soup parsing
    
    # Scrape dynamic sites
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CLASS_NAME, 'review-container'), parse_tripadvisor)
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CLASS_NAME, 'jftiEf'), parse_google)
    
    all_reviews = booking_reviews + momondo_reviews + tripadvisor_reviews + google_reviews
    save_to_csv(all_reviews)

Booking.com fetch status: 200
Booking.com fetch status: 200
Momondo fetch status: 406
Failed to fetch Momondo page 1 (status: 406)
No reviews found on TripAdvisor page 1
No next button found on TripAdvisor page 1
Saved 58 reviews to hotel_reviews___.csv


In [7]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Enhanced headers to mimic full browser request and avoid 406
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    if source == 'Momondo':
        pages = 1  # No pagination for Momondo
    for page in range(1, pages + 1):
        paginated_url = f"{url}&page={page}" if 'booking' in url.lower() else url
        response = requests.get(paginated_url, headers=HEADERS, allow_redirects=True)
        print(f"{source} fetch status: {response.status_code}")  # Log status for debugging
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} page {page} (status: {response.status_code})")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        if source == 'Momondo':
            # For Momondo, parse entire soup directly
            parsed_reviews = parse_func(soup)
            for pr in parsed_reviews:
                pr['source'] = source
                reviews.append(pr)
        else:
            review_blocks = soup.select(review_selector)
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
    return reviews

# Parse function for Booking.com (unchanged)
def parse_booking(block):
    return {
        'reviewer': block.find('span', class_='bui-avatar-block__title').text.strip() if block.find('span', class_='bui-avatar-block__title') else 'Anonymous',
        'date': block.find('p', class_='review_item_date').text.strip() if block.find('p', class_='review_item_date') else 'N/A',
        'rating': block.find('div', class_='bui-review-score__badge').text.strip() if block.find('div', class_='bui-review-score__badge') else 'N/A',
        'title': block.find('h3', class_='c-review-block__title').text.strip() if block.find('h3', class_='c-review-block__title') else 'No Title',
        'text': ' '.join([elem.text.strip() for elem in block.find_all('span', class_='c-review__body')]) if block.find_all('span', class_='c-review__body') else 'No Text'
    }

# Parse function for Momondo (refined selectors without class assumption)
def parse_momondo(soup):
    reviews = []
    pros_h3 = soup.find('h3', string=lambda text: text and 'Pros +' in text.strip())
    if pros_h3:
        pros_ul = pros_h3.find_next('ul')
        if pros_ul:
            items = [li.text.strip() for li in pros_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Pros',
                'text': '; '.join(items)
            })

    cons_h3 = soup.find('h3', string=lambda text: text and 'Cons -' in text.strip())
    if cons_h3:
        cons_ul = cons_h3.find_next('ul')
        if cons_ul:
            items = [li.text.strip() for li in cons_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Cons',
                'text': '; '.join(items)
            })
    return reviews if reviews else [{'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'No pros/cons found'}]

# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)  # Longer initial load for stability

    for page_num in range(pages):
        if source == 'Google Maps':
            try:
                reviews_panel = driver.find_element(By.CSS_SELECTOR, '.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')
                for _ in range(15):  # Further increased scrolls to load more if available
                    driver.execute_script("arguments[0].scrollBy(0, 2000);", reviews_panel)
                    time.sleep(1.5)
            except:
                print("No reviews panel found on Google Maps")

        try:
            review_blocks = driver.find_elements(*review_locator)
            if not review_blocks:
                print(f"No reviews found on {source} page {page_num + 1}")
            for block in review_blocks:
                try:
                    review = parse_func(block)
                    review['source'] = source
                    reviews.append(review)
                except Exception as e:
                    print(f"Error parsing {source} review: {e}")
            # Pagination
            if source == 'TripAdvisor':
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '.ui_button.nav.next.primary')
                    if next_button and next_button.is_displayed() and next_button.is_enabled():
                        next_button.click()
                        time.sleep(5)
                    else:
                        print(f"No next button or not enabled on TripAdvisor page {page_num + 1}")
                        break
                except:
                    print(f"No next button found on TripAdvisor page {page_num + 1}")
                    break
            else:
                break
        except Exception as e:
            print(f"Failed to process {source} page {page_num + 1}: {e}")
            break
    driver.quit()
    return reviews

# Parse function for TripAdvisor (added robustness)
def parse_tripadvisor(block):
    try:
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.info_text div:first-child')
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'
        date_elem = block.find_elements(By.CSS_SELECTOR, '.ratingDate')
        date = date_elem[0].get_attribute('title') if date_elem else 'N/A'
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating')
        rating_class = rating_elem[0].get_attribute('class') if rating_elem else ''
        rating = str(int(rating_class.split('_')[-1]) / 10) if rating_class else 'N/A'
        title_elem = block.find_elements(By.CSS_SELECTOR, '.noQuotes')
        title = title_elem[0].text.strip() if title_elem else 'No Title'
        text_elem = block.find_elements(By.CSS_SELECTOR, '.partial_entry')
        text = text_elem[0].text.strip() if text_elem else 'No Text'
        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': title,
            'text': text
        }
    except:
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}

# Parse function for Google Maps (unchanged)
def parse_google(block):
    return {
        'reviewer': block.find_element(By.CSS_SELECTOR, '.d4r55').text.strip() if block.find_elements(By.CSS_SELECTOR, '.d4r55') else 'Anonymous',
        'date': block.find_element(By.CSS_SELECTOR, '.rsqaWe').text.strip() if block.find_elements(By.CSS_SELECTOR, '.rsqaWe') else 'N/A',
        'rating': block.get_attribute('aria-label') if block.find_elements(By.CSS_SELECTOR, '.kvMYJc') else 'N/A',
        'title': 'No Title',
        'text': block.find_element(By.CSS_SELECTOR, '.wiI7pd').text.strip() if block.find_elements(By.CSS_SELECTOR, '.wiI7pd') else 'No Text'
    }

# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews__1.csv'):
    if not reviews:
        print("No reviews to save.")
        return
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, keys)
        writer.writeheader()
        writer.writerows(reviews)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    # Scrape static sites
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.review_list_new_item_block', parse_booking)
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', '', parse_momondo)  # Empty selector for soup parsing
    
    # Scrape dynamic sites
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CLASS_NAME, 'review-container'), parse_tripadvisor)
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CLASS_NAME, 'jftiEf'), parse_google)
    
    all_reviews = booking_reviews + momondo_reviews + tripadvisor_reviews + google_reviews
    save_to_csv(all_reviews)

Booking.com fetch status: 200
Booking.com fetch status: 200
Momondo fetch status: 200
No reviews found on TripAdvisor page 1
No next button found on TripAdvisor page 1
Saved 59 reviews to hotel_reviews__1.csv


In [8]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re # Pour la gestion de l'URL de Booking.com

# Enhanced headers to mimic full browser request and avoid 406
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    
    # Momondo is special, it doesn't have review blocks in the same way, and no pagination for "pros/cons"
    if source == 'Momondo':
        response = requests.get(url, headers=HEADERS, allow_redirects=True)
        print(f"{source} fetch status: {response.status_code}")
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} (status: {response.status_code})")
            return []
        soup = BeautifulSoup(response.text, 'html.parser')
        # parse_func for Momondo returns a list of reviews/items
        parsed_momondo_reviews = parse_func(soup)
        for pr in parsed_momondo_reviews:
            pr['source'] = source
            reviews.append(pr)
        return reviews

    # Standard BS scraping for paginated sites like Booking
    for page in range(1, pages + 1):
        paginated_url = url
        if 'booking' in url.lower():
            # Robust URL parameter replacement for Booking pagination
            if "page=" in paginated_url:
                paginated_url = re.sub(r"page=\d+", f"page={page}", paginated_url)
            else:
                paginated_url = f"{paginated_url}&page={page}"

        print(f"Fetching {source} page {page} from {paginated_url}")
        response = requests.get(paginated_url, headers=HEADERS, allow_redirects=True)
        print(f"{source} page {page} fetch status: {response.status_code}")
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} page {page} (status: {response.status_code})")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.select(review_selector)

        if not review_blocks and page == 1:
            print(f"No reviews found on {source} first page with selector '{review_selector}'.")
            break # No reviews at all
        elif not review_blocks:
            print(f"No more reviews on {source} page {page}.")
            break # End of pagination

        for block in review_blocks:
            try:
                review = parse_func(block)
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e} in block: {block.prettify()[:200]}...") # Log partial block for debug
        
        time.sleep(random.uniform(2, 5)) # Be polite

    return reviews

# Parse function for Booking.com (Potential updated selectors)
def parse_booking(block):
    # These selectors are examples. YOU MUST VERIFY THEM with browser inspector.
    reviewer = block.select_one('.bui-avatar-block__title') # This is for reviewer name
    if not reviewer: reviewer = block.select_one('.bui-link--light') # Fallback for reviewer

    date = block.select_one('.review_item_date') # Example
    rating = block.select_one('.bui-review-score__badge') # Example
    title = block.select_one('.c-review-block__title') # Example
    
    # Booking often has positive and negative parts
    positive_text_elem = block.select_one('.c-review__body:nth-of-type(1)') # Adjust if multiple .c-review__body
    negative_text_elem = block.select_one('.c-review__body:nth-of-type(2)')
    
    full_text = []
    if positive_text_elem: full_text.append(positive_text_elem.text.strip())
    if negative_text_elem: full_text.append(negative_text_elem.text.strip())

    return {
        'reviewer': reviewer.text.strip() if reviewer else 'Anonymous',
        'date': date.text.strip() if date else 'N/A',
        'rating': rating.text.strip() if rating else 'N/A',
        'title': title.text.strip() if title else 'No Title',
        'text': ' '.join(full_text) if full_text else 'No Text'
    }

# Parse function for Momondo (refined selectors and consistent return)
def parse_momondo(soup):
    reviews = []
    
    # Momondo tends to aggregate pros/cons, not individual reviews easily
    # It might be better to capture this as one "aggregated" review or ignore for individual sentiment analysis.
    # For now, we'll try to capture them as separate "reviews" in the list.

    pros_text = []
    pros_h3 = soup.find('h3', string=lambda text: text and 'Pros +' in text.strip())
    if pros_h3:
        pros_ul = pros_h3.find_next('ul')
        if pros_ul:
            pros_text = [li.text.strip() for li in pros_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Momondo Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Pros of Business Hotel',
                'text': '; '.join(pros_text),
                'sentiment_label': 'Positive' # Can manually label these
            })

    cons_text = []
    cons_h3 = soup.find('h3', string=lambda text: text and 'Cons -' in text.strip())
    if cons_h3:
        cons_ul = cons_h3.find_next('ul')
        if cons_ul:
            cons_text = [li.text.strip() for li in cons_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Momondo Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Cons of Business Hotel',
                'text': '; '.join(cons_text),
                'sentiment_label': 'Negative' # Can manually label these
            })
    
    if not reviews:
        reviews.append({'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'No pros/cons found', 'sentiment_label': 'Neutral'})
    
    return reviews


# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True # Run in headless mode
    # Fix for newer Chrome versions:
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={HEADERS['User-Agent']}") # Pass User-Agent to Selenium

    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        print(f"Initial page load for {source}: {url}")
        time.sleep(5)  # Longer initial load for stability

        # --- Specific logic for Google Maps & TripAdvisor ---
        if source == 'Google Maps':
            # This is the reviews panel, it's often more reliable to find it by its ARIA role or data-attributes
            # Or by its parent container that is scrollable
            # This CSS selector (.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde) might be volatile.
            # Look for a more stable element, e.g., the div containing all reviews, which has a scrollbar.
            try:
                # Find the scrollable reviews panel
                # This selector is crucial and needs to be verified on Google Maps
                reviews_panel = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')) # Verify this
                )
                print("Google Maps reviews panel found.")
                # Scroll multiple times to load all reviews
                for _ in range(20): # Increased scrolls for more reviews
                    driver.execute_script("arguments[0].scrollBy(0, 5000);", reviews_panel) # Scroll more aggressively
                    time.sleep(2) # Increased sleep for more content to load
                    # Optional: check if new reviews loaded to stop early
                print("Finished scrolling Google Maps reviews.")
            except Exception as e:
                print(f"Could not find or scroll Google Maps reviews panel: {e}")

        # TripAdvisor pagination is handled internally by Selenium loop
        
        # --- Extract reviews after loading/scrolling ---
        review_blocks = driver.find_elements(*review_locator)
        if not review_blocks:
            print(f"No review blocks found for {source} with locator {review_locator}.")
        
        # This loop will now get all currently loaded reviews after scrolling/pagination
        for block in review_blocks:
            try:
                review = parse_func(block)
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e} in block: {block.text[:100]}...") # Log partial block

        # TripAdvisor specific pagination (if not already handled by the above loop)
        if source == 'TripAdvisor':
            for page_num in range(pages): # pages here represents "next" clicks
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '.ui_button.nav.next.primary') # Verify selector
                    if next_button.is_displayed() and next_button.is_enabled():
                        next_button.click()
                        time.sleep(5) # Wait for new page to load
                        
                        # Re-find review blocks after navigating to next page
                        new_review_blocks = driver.find_elements(*review_locator)
                        if not new_review_blocks:
                            print(f"No new reviews found after clicking next on TripAdvisor page {page_num + 1}.")
                            break
                        for block in new_review_blocks:
                            try:
                                review = parse_func(block)
                                review['source'] = source
                                reviews.append(review)
                            except Exception as e:
                                print(f"Error parsing {source} review: {e} in new block: {block.text[:100]}...")
                    else:
                        print(f"No more next button or not enabled on TripAdvisor after {page_num + 1} pages.")
                        break
                except Exception as e:
                    print(f"Error with TripAdvisor pagination on page {page_num + 1}: {e}")
                    break
        
    finally:
        if driver:
            driver.quit()
    return reviews


# Parse function for TripAdvisor (using Selenium WebElement methods)
def parse_tripadvisor(block):
    # These selectors are examples. YOU MUST VERIFY THEM with browser inspector.
    try:
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.info_text div:first-child') # Example
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'
        
        date_elem = block.find_elements(By.CSS_SELECTOR, '.ratingDate') # Example
        date = date_elem[0].get_attribute('title') if date_elem else 'N/A' # Date is often in title attribute
        
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating') # Example
        rating = 'N/A'
        if rating_elem:
            # Rating is often in a class like 'bubble_50' for 5 stars, 'bubble_40' for 4 stars
            for cls in rating_elem[0].get_attribute('class').split():
                if 'bubble_' in cls:
                    rating = str(int(cls.replace('bubble_', '')) / 10.0) # Convert 'bubble_50' to '5.0'
                    break

        title_elem = block.find_elements(By.CSS_SELECTOR, '.noQuotes') # Example
        title = title_elem[0].text.strip() if title_elem else 'No Title'
        
        text_elem = block.find_elements(By.CSS_SELECTOR, '.partial_entry') # Example, might need to click "more"
        text = text_elem[0].text.strip() if text_elem else 'No Text'
        
        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': title,
            'text': text
        }
    except Exception as e:
        print(f"TripAdvisor parse error: {e}. Block text: {block.text[:100]}...")
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}

# Parse function for Google Maps (using Selenium WebElement methods)
def parse_google(block):
    # These selectors are examples. YOU MUST VERIFY THEM with browser inspector.
    try:
        # Reviewer name
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.d4r55') # Example
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'

        # Date
        date_elem = block.find_elements(By.CSS_SELECTOR, '.rsqaWe') # Example
        date = date_elem[0].text.strip() if date_elem else 'N/A'
        
        # Rating - often an aria-label on a star element or its parent
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.kvMYJc') # Example: the div containing stars
        rating = 'N/A'
        if rating_elem:
            rating_label = rating_elem[0].get_attribute('aria-label') # "Note 5 sur 5"
            if rating_label and 'sur' in rating_label:
                rating = rating_label.split(' ')[1] # Extract '5'
        
        # Full review text
        text_elem = block.find_elements(By.CSS_SELECTOR, '.wiI7pd') # Example
        text = text_elem[0].text.strip() if text_elem else 'No Text'

        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': 'No Title (Google Maps)', # Google Maps reviews typically don't have titles
            'text': text
        }
    except Exception as e:
        print(f"Google Maps parse error: {e}. Block text: {block.text[:100]}...")
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}


# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews_combined.csv'): # Changed filename
    if not reviews:
        print("No reviews to save.")
        return
    # Ensure all review dictionaries have the same keys for DictWriter
    # Create a superset of all keys found across all reviews
    all_keys = set()
    for review in reviews:
        all_keys.update(review.keys())
    
    # Fill missing keys with None for consistency
    reviews_for_csv = []
    for review in reviews:
        full_review = {key: review.get(key) for key in all_keys}
        reviews_for_csv.append(full_review)

    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, sorted(list(all_keys))) # Sort keys for consistent column order
        writer.writeheader()
        writer.writerows(reviews_for_csv)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    all_reviews = []

    print("\n--- Scraping Booking.com ---")
    # Increased pages for Booking (75 reviews/page * 20 pages = 1500 potential reviews)
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.c-review-block', parse_booking, pages=20) 
    all_reviews.extend(booking_reviews)
    
    print("\n--- Scraping Momondo.com ---")
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', '', parse_momondo, pages=1) # Momondo is one-off
    all_reviews.extend(momondo_reviews)
    
    print("\n--- Scraping TripAdvisor.com ---")
    # TripAdvisor pages count how many "next" clicks to simulate
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CSS_SELECTOR, 'div[data-reviewid]'), parse_tripadvisor, pages=10) 
    all_reviews.extend(tripadvisor_reviews)

    print("\n--- Scraping Google Maps ---")
    # Google Maps needs more scrolls than pages, so 'pages' here could mean "how many times to try and scroll and extract"
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CSS_SELECTOR, 'div.jftiEf'), parse_google, pages=1) # pages here refers to how many times to execute the scroll loop
    all_reviews.extend(google_reviews)
    
    save_to_csv(all_reviews)


--- Scraping Booking.com ---
Fetching Booking.com page 1 from https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75&page=1
Booking.com page 1 fetch status: 200
No reviews found on Booking.com first page with selector 'div.c-review-block'.

--- Scraping Momondo.com ---
Momondo fetch status: 200

--- Scraping TripAdvisor.com ---
Initial page load for TripAdvisor: https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html
No review blocks found for TripAdvisor with locator ('css selector', 'div[data-reviewid]').
Error with TripAdvisor pagination on page 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ui_button.nav.next.primary"}

In [12]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re  # Pour la gestion de l'URL de Booking.com
import random  # For polite delays

# Enhanced headers to mimic full browser request and avoid 406
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

# Function for BeautifulSoup scraping (static sites)
def scrape_with_bs(url, source, review_selector, parse_func, pages=2):
    reviews = []
    
    # Momondo is special, it doesn't have review blocks in the same way, and no pagination for "pros/cons"
    if source == 'Momondo':
        response = requests.get(url, headers=HEADERS, allow_redirects=True)
        print(f"{source} fetch status: {response.status_code}")
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} (status: {response.status_code})")
            return []
        soup = BeautifulSoup(response.text, 'html.parser')
        # parse_func for Momondo returns a list of reviews/items
        parsed_momondo_reviews = parse_func(soup)
        for pr in parsed_momondo_reviews:
            pr['source'] = source
            reviews.append(pr)
        return reviews
    
    # Standard BS scraping for paginated sites like Booking
    for page in range(1, pages + 1):
        paginated_url = url
        if 'booking' in url.lower():
            # Robust URL parameter replacement for Booking pagination
            if "page=" in paginated_url:
                paginated_url = re.sub(r"page=\d+", f"page={page}", paginated_url)
            else:
                paginated_url = f"{paginated_url}&page={page}"
        print(f"Fetching {source} page {page} from {paginated_url}")
        response = requests.get(paginated_url, headers=HEADERS, allow_redirects=True)
        print(f"{source} page {page} fetch status: {response.status_code}")
        if response.status_code not in [200, 301, 302]:
            print(f"Failed to fetch {source} page {page} (status: {response.status_code})")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.select(review_selector)
        if not review_blocks and page == 1:
            print(f"No reviews found on {source} first page with selector '{review_selector}'.")
            break  # No reviews at all
        elif not review_blocks:
            print(f"No more reviews on {source} page {page}.")
            break  # End of pagination
        for block in review_blocks:
            try:
                review = parse_func(block)
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e} in block: {block.prettify()[:200]}...")  # Log partial block for debug
        
        time.sleep(random.uniform(2, 5))  # Be polite
    return reviews

# Parse function for Booking.com (Updated selectors based on site inspection)
def parse_booking(block):
    # Updated selectors from site structure
    reviewer = block.select_one('span.bui-avatar-block__title')  # Reviewer name
    date = block.select_one('span.c-review-block__date')  # Date
    rating = block.select_one('div.bui-review-score__badge')  # Rating
    title = block.select_one('h3.c-review-block__title')  # Title
    
    # Booking often has positive and negative parts
    positive_text_elem = block.select('span.c-review__body--original')
    negative_text_elem = block.select('span.c-review__body')
    
    full_text = []
    if positive_text_elem:
        full_text.extend([elem.text.strip() for elem in positive_text_elem])
    if negative_text_elem:
        full_text.extend([elem.text.strip() for elem in negative_text_elem if elem not in positive_text_elem])  # Avoid duplicates
    return {
        'reviewer': reviewer.text.strip() if reviewer else 'Anonymous',
        'date': date.text.strip() if date else 'N/A',
        'rating': rating.text.strip() if rating else 'N/A',
        'title': title.text.strip() if title else 'No Title',
        'text': ' '.join(full_text) if full_text else 'No Text'
    }

# Parse function for Momondo (refined to handle h3 or h6 headings)
def parse_momondo(soup):
    reviews = []
    
    # Flexible heading search for Pros +
    pros_heading = soup.find(lambda tag: tag.name in ['h3', 'h6'] and 'Pros +' in tag.text.strip())
    if pros_heading:
        pros_ul = pros_heading.find_next('ul')
        if pros_ul:
            items = [li.text.strip() for li in pros_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Momondo Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Pros of Business Hotel',
                'text': '; '.join(items),
                'sentiment_label': 'Positive'  # Can manually label these
            })
    
    # Flexible heading search for Cons -
    cons_heading = soup.find(lambda tag: tag.name in ['h3', 'h6'] and 'Cons -' in tag.text.strip())
    if cons_heading:
        cons_ul = cons_heading.find_next('ul')
        if cons_ul:
            items = [li.text.strip() for li in cons_ul.find_all('li')]
            reviews.append({
                'reviewer': 'Momondo Aggregated',
                'date': 'N/A',
                'rating': 'N/A',
                'title': 'Cons of Business Hotel',
                'text': '; '.join(items),
                'sentiment_label': 'Negative'  # Can manually label these
            })
    
    if not reviews:
        reviews.append({'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'No pros/cons found', 'sentiment_label': 'Neutral'})
    
    return reviews

# Function for Selenium scraping (dynamic sites)
def scrape_with_selenium(url, source, review_locator, parse_func, pages=2):
    reviews = []
    options = Options()
    options.headless = True  # Run in headless mode
    # Fix for newer Chrome versions:
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={HEADERS['User-Agent']}")  # Pass User-Agent to Selenium
    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        print(f"Initial page load for {source}: {url}")
        time.sleep(5)  # Longer initial load for stability
        # --- Specific logic for Google Maps & TripAdvisor ---
        if source == 'Google Maps':
            try:
                # Find the scrollable reviews panel
                reviews_panel = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))  # Verify this
                )
                print("Google Maps reviews panel found.")
                # Scroll multiple times to load all reviews
                for _ in range(20):  # Increased scrolls for more reviews
                    driver.execute_script("arguments[0].scrollBy(0, 5000);", reviews_panel)  # Scroll more aggressively
                    time.sleep(2)  # Increased sleep for more content to load
                    # Optional: check if new reviews loaded to stop early
                print("Finished scrolling Google Maps reviews.")
            except Exception as e:
                print(f"Could not find or scroll Google Maps reviews panel: {e}")
        # TripAdvisor specific pagination (if not already handled by the above loop)
        if source == 'TripAdvisor':
            for page_num in range(pages):  # pages here represents "next" clicks
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '.ui_button.nav.next.primary')  # Verify selector
                    if next_button.is_displayed() and next_button.is_enabled():
                        next_button.click()
                        time.sleep(5)  # Wait for new page to load
                        
                        # Re-find review blocks after navigating to next page
                        new_review_blocks = driver.find_elements(*review_locator)
                        if not new_review_blocks:
                            print(f"No new reviews found after clicking next on TripAdvisor page {page_num + 1}.")
                            break
                        for block in new_review_blocks:
                            try:
                                review = parse_func(block)
                                review['source'] = source
                                reviews.append(review)
                            except Exception as e:
                                print(f"Error parsing {source} review: {e} in new block: {block.text[:100]}...")
                    else:
                        print(f"No more next button or not enabled on TripAdvisor after {page_num + 1} pages.")
                        break
                except Exception as e:
                    print(f"Error with TripAdvisor pagination on page {page_num + 1}: {e}")
                    break
        
        # --- Extract reviews after loading/scrolling ---
        review_blocks = driver.find_elements(*review_locator)
        if not review_blocks:
            print(f"No review blocks found for {source} with locator {review_locator}.")
        
        # This loop will now get all currently loaded reviews after scrolling/pagination
        for block in review_blocks:
            try:
                review = parse_func(block)
                review['source'] = source
                reviews.append(review)
            except Exception as e:
                print(f"Error parsing {source} review: {e} in block: {block.text[:100]}...")  # Log partial block
        
    finally:
        if driver:
            driver.quit()
    return reviews

# Parse function for TripAdvisor (using Selenium WebElement methods)
def parse_tripadvisor(block):
    try:
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.info_text div:first-child')  # Example
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'
        
        date_elem = block.find_elements(By.CSS_SELECTOR, '.ratingDate')  # Example
        date = date_elem[0].get_attribute('title') if date_elem else 'N/A'  # Date is often in title attribute
        
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.ui_bubble_rating')  # Example
        rating = 'N/A'
        if rating_elem:
            # Rating is often in a class like 'bubble_50' for 5 stars, 'bubble_40' for 4 stars
            for cls in rating_elem[0].get_attribute('class').split():
                if 'bubble_' in cls:
                    rating = str(int(cls.replace('bubble_', '')) / 10.0)  # Convert 'bubble_50' to '5.0'
                    break
        title_elem = block.find_elements(By.CSS_SELECTOR, '.noQuotes')  # Example
        title = title_elem[0].text.strip() if title_elem else 'No Title'
        
        text_elem = block.find_elements(By.CSS_SELECTOR, '.partial_entry')  # Example, might need to click "more"
        text = text_elem[0].text.strip() if text_elem else 'No Text'
        
        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': title,
            'text': text
        }
    except Exception as e:
        print(f"TripAdvisor parse error: {e}. Block text: {block.text[:100]}...")
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}

# Parse function for Google Maps (using Selenium WebElement methods)
def parse_google(block):
    try:
        # Reviewer name
        reviewer_elem = block.find_elements(By.CSS_SELECTOR, '.d4r55')  # Example
        reviewer = reviewer_elem[0].text.strip() if reviewer_elem else 'Anonymous'
        # Date
        date_elem = block.find_elements(By.CSS_SELECTOR, '.rsqaWe')  # Example
        date = date_elem[0].text.strip() if date_elem else 'N/A'
        
        # Rating - often an aria-label on a star element or its parent
        rating_elem = block.find_elements(By.CSS_SELECTOR, '.kvMYJc')  # Example: the div containing stars
        rating = 'N/A'
        if rating_elem:
            rating_label = rating_elem[0].get_attribute('aria-label')  # "Note 5 sur 5"
            if rating_label and 'sur' in rating_label:
                rating = rating_label.split(' ')[1]  # Extract '5'
        
        # Full review text
        text_elem = block.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # Example
        text = text_elem[0].text.strip() if text_elem else 'No Text'
        return {
            'reviewer': reviewer,
            'date': date,
            'rating': rating,
            'title': 'No Title (Google Maps)',  # Google Maps reviews typically don't have titles
            'text': text
        }
    except Exception as e:
        print(f"Google Maps parse error: {e}. Block text: {block.text[:100]}...")
        return {'reviewer': 'N/A', 'date': 'N/A', 'rating': 'N/A', 'title': 'No Data', 'text': 'Parse failed'}

# Save to CSV
def save_to_csv(reviews, filename='hotel_reviews_combined11.csv'):  # Changed filename
    if not reviews:
        print("No reviews to save.")
        return
    # Ensure all review dictionaries have the same keys for DictWriter
    # Create a superset of all keys found across all reviews
    all_keys = set()
    for review in reviews:
        all_keys.update(review.keys())
    
    # Fill missing keys with None for consistency
    reviews_for_csv = []
    for review in reviews:
        full_review = {key: review.get(key) for key in all_keys}
        reviews_for_csv.append(full_review)
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, sorted(list(all_keys)))  # Sort keys for consistent column order
        writer.writeheader()
        writer.writerows(reviews_for_csv)
    print(f"Saved {len(reviews)} reviews to {filename}")

# Main execution
if __name__ == "__main__":
    # URLs
    booking_url = "https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75"
    momondo_url = "https://www.momondo.com/hotels/tunis-tunis-governorate/Business-Hotel-Tunis.mhd2417013.ksp"
    tripadvisor_url = "https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html"
    google_url = "https://www.google.com/maps/place/Business+H%C3%B4tel/@36.818325,10.1820211,17z/data=!4m8!3m7!1s0x12fd3489b5a4f4e1:0xdda07f445b5b03cd!8m2!3d36.818325!4d10.184596!9m1!1b1!16s%2Fg%2F11bwn563xh?entry=ttu"
    
    all_reviews = []
    print("\n--- Scraping Booking.com ---")
    # Increased pages for Booking (75 reviews/page * 20 pages = 1500 potential reviews)
    booking_reviews = scrape_with_bs(booking_url, 'Booking.com', 'div.review_list_new_item_block', parse_booking, pages=20)
    all_reviews.extend(booking_reviews)
    
    print("\n--- Scraping Momondo.com ---")
    momondo_reviews = scrape_with_bs(momondo_url, 'Momondo', '', parse_momondo, pages=1)  # Momondo is one-off
    all_reviews.extend(momondo_reviews)
    
    print("\n--- Scraping TripAdvisor.com ---")
    # TripAdvisor pages count how many "next" clicks to simulate
    tripadvisor_reviews = scrape_with_selenium(tripadvisor_url, 'TripAdvisor', (By.CLASS_NAME, 'review-container'), parse_tripadvisor, pages=10)
    all_reviews.extend(tripadvisor_reviews)
    
    print("\n--- Scraping Google Maps ---")
    # Google Maps needs more scrolls than pages, so 'pages' here could mean "how many times to try and scroll and extract"
    google_reviews = scrape_with_selenium(google_url, 'Google Maps', (By.CLASS_NAME, 'jftiEf'), parse_google, pages=1)  # pages here refers to how many times to execute the scroll loop
    all_reviews.extend(google_reviews)
    
    save_to_csv(all_reviews)


--- Scraping Booking.com ---
Fetching Booking.com page 1 from https://www.booking.com/reviews/tn/hotel/business.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYAQe4AQfIAQzYAQHoAQH4AQKIAgGoAgO4ApvZoMoGwAIB0gIkY2Q5ZGY5ZjMtZWEyNi00NzE5LWI5NjgtMzY4N2E4N2U3M2Q32AIG4AIB&sid=32b2429f18b4624fc09d32f51f4441cc&customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&rows=75&page=1
Booking.com page 1 fetch status: 200
No reviews found on Booking.com first page with selector 'div.review_list_new_item_block'.

--- Scraping Momondo.com ---
Momondo fetch status: 200

--- Scraping TripAdvisor.com ---
Initial page load for TripAdvisor: https://www.tripadvisor.com/Hotel_Review-g293758-d8767447-Reviews-Business_Hotel_Tunis-Tunis_Tunis_Governorate.html
Error with TripAdvisor pagination on page 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ui_button.nav.next.primary"}
  (Session info: chrome=142.0.7444.176); For documentation on this error, pleas